Extractive Summarization Section

In [1]:
import json
import pandas as pd
import numpy as np
from helper_functions import *

In [14]:
clusters = pd.read_csv('./summarization_cluster_analysis.csv')

In [15]:
df = pd.read_csv('./Telehealth.csv')

In [16]:
clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  44 non-null     int64  
 1   Unnamed: 0.1                44 non-null     int64  
 2   year                        44 non-null     int64  
 3   class                       44 non-null     object 
 4   cleaned_pca_component_1     44 non-null     float64
 5   cleaned_pca_component_2     44 non-null     float64
 6   cleaned_pca_dbscan_class    44 non-null     int64  
 7   cleaned_tsne_dim_1          44 non-null     float64
 8   cleaned_tsne_dim_2          44 non-null     float64
 9   cleaned_tsne_dbscan_class   44 non-null     int64  
 10  lemma_pca_component_1       44 non-null     float64
 11  lemma_pca_component_2       44 non-null     float64
 12  lemma_pca_dbscan_class      44 non-null     int64  
 13  lemma_tsne_dim_1            44 non-nu

In [17]:
df.head(1)

,Journal Title,Article Title,Date Published,Authors,Abstract,Keywords,Citation,Content,Content_Length,Abstract_Length,Parsed_Keywords,Parsed_Keywords_Length,Subfield,et_al_Count,Stopwords_Lemma_Longform_Clean_Content,Clean_Content,Classification
0,Psychological Services,The Effectiveness of Telepsychology With Veter...,2021,Michael J. McClellan; Richard Osbaldiston; Ron...,Veterans face a variety of stressors due to th...,"KEYWORDS:\n\ntelepsychology, meta-analysis, ve...","McClellan, M. J., Osbaldiston, R., Wu, R., Yea...",Veterans face a variety of stressors related t...,37477,2411,"['telepsychology,', 'meta-analysis,', 'veteran...",5,Clinical & Counseling Psychology,50,veteran face variety stressor relate military ...,veteran face variety stressor relate military ...,Covid


In [18]:
merged = df.merge(clusters, on= 'Article Title')

In [19]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 43
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Journal Title_x                         44 non-null     object 
 1   Article Title                           44 non-null     object 
 2   Date Published                          44 non-null     int64  
 3   Authors                                 44 non-null     object 
 4   Abstract_x                              39 non-null     object 
 5   Keywords_x                              42 non-null     object 
 6   Citation                                44 non-null     object 
 7   Content                                 44 non-null     object 
 8   Content_Length_x                        44 non-null     int64  
 9   Abstract_Length                         44 non-null     int64  
 10  Parsed_Keywords                         44 non-null     object 


In [20]:
# Drop everything we don't need to focus on the summaries:

summ_df = merged[['Abstract_x', 'Content', 'lemma_tsne_dbscan_class']].copy()

Create the three cluster corpora, exclude the three papers in the unassigned -1 category

In [22]:
summ_df_0 = summ_df[summ_df['lemma_tsne_dbscan_class']==0]
summ_df_1 = summ_df[summ_df['lemma_tsne_dbscan_class']==1]
summ_df_2 = summ_df[summ_df['lemma_tsne_dbscan_class']==2]

In [23]:
def create_text(df):
  text = ''
  abstracts = ''
  for i in df['Content']:
    text = text + i + ' '
  for j in df['Abstract_x']:
    try:
      abstracts = abstracts + j + ' '
    except:
      continue
  return text, abstracts

Corpus0, abstracts0 = create_text(summ_df_0)
Corpus1, abstracts1 = create_text(summ_df_1)
Corpus2, abstracts2 = create_text(summ_df_2)
Corpus0

"Veterans face a variety of stressors related to their military service (Institute of Medicine, 2014; Tanielian et al., 2008). Most veterans who return from military conflict do not have a mental health condition, but all veterans “experience a period of readjustment as they reintegrate into life with family, friends, and community” (Substance Abuse and Mental Health Services Administration [SAMHSA], 2012, p. 1; Tanielian et al., 2008). Those individuals must juggle a variety of sometimes conflicting family and military-related responsibilities, learn to readjust to environments where behaviors associated with a constant state of readiness are no longer needed, and process combat-related exposure that may collectively contribute to problems for both veterans and their families (Institute of Medicine, 2014; SAMHSA, 2012).Veterans who are exposed to combat are more likely to report normal reactions to stress including depression, hopelessness, insomnia, nightmares, feelings of rejection,

<h1>GENSIM Summary Section:

In [24]:
from gensim.summarization import summarize

In [26]:
corpora = [Corpus0, Corpus1, Corpus2]
new_table = []
for i,text in enumerate(corpora):
  # print(summarize(text, word_count=150))
  new_table.append(summarize(text,word_count = 150))
Summaries = pd.DataFrame(new_table, columns=['Summaries'])
Summaries['Abstracts'] = [abstracts0, abstracts1, abstracts2]

In [27]:
Corpus0

"Veterans face a variety of stressors related to their military service (Institute of Medicine, 2014; Tanielian et al., 2008). Most veterans who return from military conflict do not have a mental health condition, but all veterans “experience a period of readjustment as they reintegrate into life with family, friends, and community” (Substance Abuse and Mental Health Services Administration [SAMHSA], 2012, p. 1; Tanielian et al., 2008). Those individuals must juggle a variety of sometimes conflicting family and military-related responsibilities, learn to readjust to environments where behaviors associated with a constant state of readiness are no longer needed, and process combat-related exposure that may collectively contribute to problems for both veterans and their families (Institute of Medicine, 2014; SAMHSA, 2012).Veterans who are exposed to combat are more likely to report normal reactions to stress including depression, hopelessness, insomnia, nightmares, feelings of rejection,

In [28]:
# summarize(Corpus0,word_count = 100)

<h1>BLEU Scoring Section:

In [29]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [30]:
bleu_score_dict = {}

for row in range(len(Summaries)):
    # Using (1,0,0,0) weights to only search for unigram similarities since beyond that the scores were infintesimal
    
    bleu_score = sentence_bleu(Summaries.Abstracts[row], Summaries.Summaries[row])
    print(bleu_score)
    bleu_score_dict[row] = bleu_score
# Create a column for the bleu score from the blue_score_dict:
Summaries['Bleu Score'] = Summaries.from_dict(bleu_score_dict.values())

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4519932985080892
0.4493898816383837
0.4386913376508308


In [31]:
Summaries.head()

,Summaries,Abstracts,Bleu Score
0,Given the consequences of these barriers (e.g....,Veterans face a variety of stressors due to th...,0.451993
1,"Furthermore, while public Internet access is n...",Many university training clinics are facing nu...,0.449390
2,"In this sense, Rosen, Glassman, and Morland’s ...",Although the medical impacts of COVID-19 are n...,0.438691


<h1>Rouge Scoring Section:

In [32]:
!pip install rouge

In [33]:
from rouge import Rouge 

rouge_scorer = Rouge().get_scores

In [34]:
rouge_scores_dict = {}

for row in range(len(Summaries)):
    
    rouge_scores = rouge_scorer(Summaries.Summaries[row],Summaries.Abstracts[row])
    # rouge_scores = pd.DataFrame.from_dict(rouge_scores)
    print(rouge_scores)
    rouge_scores_dict[row] = rouge_scores

[{'rouge-1': {'f': 0.05050297610113628, 'p': 0.6949152542372882, 'r': 0.0262036642522369}, 'rouge-2': {'f': 0.01601971587743639, 'p': 0.2215909090909091, 'r': 0.008310249307479225}, 'rouge-l': {'f': 0.0762358532972738, 'p': 0.512, 'r': 0.04118404118404118}}]
[{'rouge-1': {'f': 0.09748259058308416, 'p': 0.48404255319148937, 'r': 0.054198927933293624}, 'rouge-2': {'f': 0.02144771937533671, 'p': 0.10695187165775401, 'r': 0.011918951132300357}, 'rouge-l': {'f': 0.11878787593796884, 'p': 0.34507042253521125, 'r': 0.07174231332357248}}]
[{'rouge-1': {'f': 0.07686980507220634, 'p': 0.7115384615384616, 'r': 0.04062957540263543}, 'rouge-2': {'f': 0.029799028782568786, 'p': 0.27741935483870966, 'r': 0.015745148297326986}, 'rouge-l': {'f': 0.11982378666919213, 'p': 0.5714285714285714, 'r': 0.06692913385826772}}]


In [35]:
cluster = 0
for i in rouge_scores_dict.values():
  print(i)
  x = 'rouescores' + str(cluster)
  x = pd.DataFrame.from_dict(i)
  cluster +=1

[{'rouge-1': {'f': 0.05050297610113628, 'p': 0.6949152542372882, 'r': 0.0262036642522369}, 'rouge-2': {'f': 0.01601971587743639, 'p': 0.2215909090909091, 'r': 0.008310249307479225}, 'rouge-l': {'f': 0.0762358532972738, 'p': 0.512, 'r': 0.04118404118404118}}]
[{'rouge-1': {'f': 0.09748259058308416, 'p': 0.48404255319148937, 'r': 0.054198927933293624}, 'rouge-2': {'f': 0.02144771937533671, 'p': 0.10695187165775401, 'r': 0.011918951132300357}, 'rouge-l': {'f': 0.11878787593796884, 'p': 0.34507042253521125, 'r': 0.07174231332357248}}]
[{'rouge-1': {'f': 0.07686980507220634, 'p': 0.7115384615384616, 'r': 0.04062957540263543}, 'rouge-2': {'f': 0.029799028782568786, 'p': 0.27741935483870966, 'r': 0.015745148297326986}, 'rouge-l': {'f': 0.11982378666919213, 'p': 0.5714285714285714, 'r': 0.06692913385826772}}]


In [36]:
scores = []
frames = []
for i, k in rouge_scores_dict.items():
  print(k[0])
  for score, d in k[0].items():
    scores.append(score)
    frames.append(pd.DataFrame.from_dict(d, orient='index'))

rouge = pd.concat(frames, keys=['Rouge 1 Cluster 0', 'Rouge 2 Cluster 0', 'Rouge l Cluster 0', 
                        'Rouge 1 Cluster 1', 'Rouge 2 Cluster 1', 'Rouge l Cluster 1',
                        'Rouge 1 Cluster 2', 'Rouge 2 Cluster 2', 'Rouge l Cluster 2'
                        ])

{'rouge-1': {'f': 0.05050297610113628, 'p': 0.6949152542372882, 'r': 0.0262036642522369}, 'rouge-2': {'f': 0.01601971587743639, 'p': 0.2215909090909091, 'r': 0.008310249307479225}, 'rouge-l': {'f': 0.0762358532972738, 'p': 0.512, 'r': 0.04118404118404118}}
{'rouge-1': {'f': 0.09748259058308416, 'p': 0.48404255319148937, 'r': 0.054198927933293624}, 'rouge-2': {'f': 0.02144771937533671, 'p': 0.10695187165775401, 'r': 0.011918951132300357}, 'rouge-l': {'f': 0.11878787593796884, 'p': 0.34507042253521125, 'r': 0.07174231332357248}}
{'rouge-1': {'f': 0.07686980507220634, 'p': 0.7115384615384616, 'r': 0.04062957540263543}, 'rouge-2': {'f': 0.029799028782568786, 'p': 0.27741935483870966, 'r': 0.015745148297326986}, 'rouge-l': {'f': 0.11982378666919213, 'p': 0.5714285714285714, 'r': 0.06692913385826772}}


In [37]:
rouge.to_csv('./rouge scores.csv')

In [38]:
Summaries.head()

,Summaries,Abstracts,Bleu Score
0,Given the consequences of these barriers (e.g....,Veterans face a variety of stressors due to th...,0.451993
1,"Furthermore, while public Internet access is n...",Many university training clinics are facing nu...,0.449390
2,"In this sense, Rosen, Glassman, and Morland’s ...",Although the medical impacts of COVID-19 are n...,0.438691


In [39]:
Summaries = Summaries[['Summaries', 'Bleu Score']]

In [40]:
Summaries.to_csv('./Extractive Summaries.csv')